In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import warnings

In [2]:
train = pd.read_csv('prediction_log/train_log.csv')
test = pd.read_csv('prediction_log/test_log.csv')

In [200]:
new_train = train
new_test = test

In [201]:
new_train.drop(columns=['enroll_id', 'object'], inplace=True)
new_test.drop(columns=['enroll_id', 'object'], inplace=True)

In [202]:
# Substitute various actions with category-type actions
video_action = ['seek_video','play_video','pause_video','stop_video','load_video']
problem_action = ['problem_get','problem_check','problem_save','reset_problem','problem_check_correct', 'problem_check_incorrect']
forum_action = ['create_thread','create_comment','delete_thread','delete_comment']
click_action = ['click_info','click_courseware','click_about','click_forum','click_progress']
close_action = ['close_courseware']

video = ['video']*len(video_action)
problem = ['problem']*len(problem_action)
forum = ['forum']*len(forum_action)
click = ['click']*len(click_action)
close = ['close']*len(close_action)

video_dict = dict(zip(video_action, video))
problem_dict = dict(zip(problem_action, problem))
forum_dict = dict(zip(forum_action, forum))
click_dict = dict(zip(click_action, click))
close_dict = dict(zip(close_action, close))

new_train['action'].replace(video_dict, inplace=True)
new_train['action'].replace(problem_dict, inplace=True)
new_train['action'].replace(forum_dict, inplace=True)
new_train['action'].replace(click_dict, inplace=True)
new_train['action'].replace(close_dict, inplace=True)

new_test['action'].replace(video_dict, inplace=True)
new_test['action'].replace(problem_dict, inplace=True)
new_test['action'].replace(forum_dict, inplace=True)
new_test['action'].replace(click_dict, inplace=True)
new_test['action'].replace(close_dict, inplace=True)


### Retrieve time spent on courses from join with course_info dataframe

In [203]:
courseinfo = pd.read_csv('course_info.csv', index_col='id')

In [204]:
new_train = new_train.merge(courseinfo, on='course_id')
new_test = new_test.merge(courseinfo, on='course_id')

In [205]:
new_train.drop(columns=['end', 'course_type', 'category'], inplace=True)
new_test.drop(columns=['end', 'course_type', 'category'], inplace=True)

In [207]:
new_train['time']= pd.to_datetime(new_train['time'])
new_train['start']= pd.to_datetime(new_train['start'])
new_test['time']= pd.to_datetime(new_test['time'])
new_test['start']= pd.to_datetime(new_test['start'])

In [208]:
new_train['week'] = (new_train['time'] - new_train['start']).div(7)
new_test['week'] = (new_test['time'] - new_test['start']).div(7)
def to_weeks(x):
    return int(str(x)[0]) + 1
new_train['week'] = new_train['week'].transform(to_weeks)
new_test['week'] = new_test['week'].transform(to_weeks)

In [209]:
new_train.drop(columns=['start'], inplace=True)
new_test.drop(columns=['start'], inplace=True)

In [210]:
new_train

,username,course_id,session_id,action,time,week
0,5981,course-v1:TsinghuaX+70800232X+2015_T2,d8a9b787fa69063c34c73b9c29190b1c,click,2015-09-27 15:42:59,1
1,5981,course-v1:TsinghuaX+70800232X+2015_T2,d8a9b787fa69063c34c73b9c29190b1c,click,2015-09-27 15:43:12,1
2,1544995,course-v1:TsinghuaX+70800232X+2015_T2,2f02b86eb3ea2cbf0be11385a8dc62e5,video,2015-10-19 19:37:42,4
3,1544995,course-v1:TsinghuaX+70800232X+2015_T2,2f02b86eb3ea2cbf0be11385a8dc62e5,video,2015-10-19 19:33:27,4
4,1544995,course-v1:TsinghuaX+70800232X+2015_T2,2f02b86eb3ea2cbf0be11385a8dc62e5,video,2015-10-19 19:33:30,4
...,...,...,...,...,...,...
29165535,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,dff9476c7f05f26a69a06506fe471668,click,2016-04-01 21:04:38,5
29165536,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,dff9476c7f05f26a69a06506fe471668,problem,2016-04-01 21:04:38,5
29165537,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,dff9476c7f05f26a69a06506fe471668,problem,2016-04-01 21:04:38,5
29165538,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,dff9476c7f05f26a69a06506fe471668,problem,2016-04-01 21:04:38,5


In [211]:
new_train['time_spent']= (new_train.groupby(by=['username', 'course_id', 'week', 'action', 'session_id'])['time'].transform('max')-
             new_train.groupby(by=['username', 'course_id', 'week', 'action', 'session_id'])['time'].transform('min'))
new_test['time_spent']= (new_test.groupby(by=['username', 'course_id', 'week', 'action', 'session_id'])['time'].transform('max')-
             new_test.groupby(by=['username', 'course_id', 'week', 'action', 'session_id'])['time'].transform('min'))

In [212]:
new_test

,username,course_id,session_id,action,time,week,time_spent
0,1520977,course-v1:TsinghuaX+70800232X+2015_T2,5f421f644193c2d48c84df42aaf7e48b,video,2015-10-15 22:14:11,3,0 days 01:19:46
1,1520977,course-v1:TsinghuaX+70800232X+2015_T2,5f421f644193c2d48c84df42aaf7e48b,video,2015-10-15 22:43:35,3,0 days 01:19:46
2,1520977,course-v1:TsinghuaX+70800232X+2015_T2,5f421f644193c2d48c84df42aaf7e48b,video,2015-10-15 22:43:40,3,0 days 01:19:46
3,1520977,course-v1:TsinghuaX+70800232X+2015_T2,5f421f644193c2d48c84df42aaf7e48b,video,2015-10-15 22:55:38,3,0 days 01:19:46
4,1520977,course-v1:TsinghuaX+70800232X+2015_T2,5f421f644193c2d48c84df42aaf7e48b,video,2015-10-15 22:55:38,3,0 days 01:19:46
...,...,...,...,...,...,...,...
12944857,2513464,course-v1:TsinghuaX+AP000001X+2016_T1,b12bc83e13cd943cf61bf78f09c72158,problem,2016-03-19 21:37:15,3,0 days 00:23:04
12944858,2513464,course-v1:TsinghuaX+AP000001X+2016_T1,b12bc83e13cd943cf61bf78f09c72158,problem,2016-03-19 21:37:22,3,0 days 00:23:04
12944859,2513464,course-v1:TsinghuaX+AP000001X+2016_T1,b12bc83e13cd943cf61bf78f09c72158,problem,2016-03-19 21:37:37,3,0 days 00:23:04
12944860,2513464,course-v1:TsinghuaX+AP000001X+2016_T1,7eca0904ae14dc8af809c0362632dd8e,click,2016-03-19 19:24:44,3,0 days 00:02:12


In [213]:
new_train.drop(columns=['time'], inplace=True)
new_test.drop(columns=['time'], inplace=True)

In [214]:
new_train

,username,course_id,session_id,action,week,time_spent
0,5981,course-v1:TsinghuaX+70800232X+2015_T2,d8a9b787fa69063c34c73b9c29190b1c,click,1,0 days 00:00:13
1,5981,course-v1:TsinghuaX+70800232X+2015_T2,d8a9b787fa69063c34c73b9c29190b1c,click,1,0 days 00:00:13
2,1544995,course-v1:TsinghuaX+70800232X+2015_T2,2f02b86eb3ea2cbf0be11385a8dc62e5,video,4,0 days 00:05:25
3,1544995,course-v1:TsinghuaX+70800232X+2015_T2,2f02b86eb3ea2cbf0be11385a8dc62e5,video,4,0 days 00:05:25
4,1544995,course-v1:TsinghuaX+70800232X+2015_T2,2f02b86eb3ea2cbf0be11385a8dc62e5,video,4,0 days 00:05:25
...,...,...,...,...,...,...
29165535,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,dff9476c7f05f26a69a06506fe471668,click,5,0 days 00:16:19
29165536,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,dff9476c7f05f26a69a06506fe471668,problem,5,0 days 00:00:09
29165537,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,dff9476c7f05f26a69a06506fe471668,problem,5,0 days 00:00:09
29165538,2659552,course-v1:TsinghuaX+AP000001X+2016_T1,dff9476c7f05f26a69a06506fe471668,problem,5,0 days 00:00:09


In [215]:
new_train['count'] = len(new_train)*[1]
new_test['count'] = len(new_test)*[1]

In [216]:
new_train = new_train.groupby(by=['username', 'course_id', 'action', 'week', 'session_id', 'time_spent']).count().reset_index()
new_test = new_test.groupby(by=['username', 'course_id', 'action', 'week', 'session_id', 'time_spent']).count().reset_index()

In [217]:
new_train

,username,course_id,action,week,session_id,time_spent,count
0,5,TsinghuaX/60510102X/_,close,1,c4d05fdd4af11ea45bb4623b2b6fa2c4,0 days 00:00:00,1
1,5,TsinghuaX/60510102X/_,video,1,c4d05fdd4af11ea45bb4623b2b6fa2c4,0 days 00:00:26,10
2,5,TsinghuaX/70240183x/2015_T2,close,1,6ff66b03510e0dc515ea4637216f1ad3,0 days 03:53:55,3
3,5,TsinghuaX/70240183x/2015_T2,problem,1,6ff66b03510e0dc515ea4637216f1ad3,0 days 00:00:00,1
4,5,TsinghuaX/70240183x/2015_T2,video,1,6ff66b03510e0dc515ea4637216f1ad3,0 days 00:23:04,39
...,...,...,...,...,...,...,...
1100579,7023562,course-v1:TsinghuaX+AP000008X+2017T1,problem,2,e7cc94567b145a3444a1b7abb3d7ad35,0 days 04:41:55,10
1100580,7023562,course-v1:TsinghuaX+AP000008X+2017T1,video,2,e7cc94567b145a3444a1b7abb3d7ad35,0 days 07:06:32,228
1100581,7036067,course-v1:TsinghuaX+AP000008X+2017T1,click,3,faa7749b823fbdf69fd3a752598919fa,0 days 00:02:19,7
1100582,7036067,course-v1:TsinghuaX+AP000008X+2017T1,problem,3,653c4e2bd26b9f852f0da0377081baca,0 days 00:00:50,8


In [227]:
new_test

,username,course_id,action,week,session_id,time_spent,count
0,5,course-v1:Tsinghua+20150001+2015_T2,click,1,3d52b49b694798fea515fef88c48fe1d,2 days 20:04:59,14
1,5,course-v1:Tsinghua+20150001+2015_T2,close,1,3d52b49b694798fea515fef88c48fe1d,0 days 00:07:37,3
2,5,course-v1:Tsinghua+20150001+2015_T2,video,1,3d52b49b694798fea515fef88c48fe1d,0 days 00:00:42,12
3,5,course-v1:TsinghuaX+00740123X+2016_T1,click,5,7a4aa45ee44784b026bcb95feacd5685,0 days 02:35:51,2
4,5,course-v1:TsinghuaX+00740123X+2016_T1,video,5,7a4aa45ee44784b026bcb95feacd5685,0 days 02:35:49,5
...,...,...,...,...,...,...,...
473711,7006106,course-v1:TsinghuaX+AP000008X+2017T1,video,2,41e7f9b1fb8c37493ccce0a520388d50,0 days 01:28:45,9
473712,7008105,course-v1:TsinghuaX+AP000008X+2017T1,click,1,11b8b9b4c31be1b65dbd17b919d17b7d,0 days 00:00:00,1
473713,7008105,course-v1:TsinghuaX+AP000008X+2017T1,problem,1,de27e11dd08aa456ec21370bae3c99b1,0 days 00:05:42,3
473714,7008105,course-v1:TsinghuaX+AP000008X+2017T1,video,1,11b8b9b4c31be1b65dbd17b919d17b7d,0 days 00:00:00,1


### Save dataframes to disk

In [ ]:
# Save pivot tables
new_train.to_csv('dataset/new_train.csv')
new_test.to_csv('dataset/new_test.csv')

# From now on it is only useful for Prophet

In [219]:
new_train_prophet = new_train.drop(columns=['session_id', 'time_spent'])
new_train_prophet = new_train_prophet.groupby(by=['username', 'course_id', 'action', 'week']).sum().reset_index()
new_test_prophet = new_test.drop(columns=['session_id', 'time_spent'])
new_test_prophet = new_test_prophet.groupby(by=['username', 'course_id', 'action', 'week']).sum().reset_index()

In [220]:
new_train_prophet_pivot = new_train_prophet.pivot(index=['username', 'course_id', 'action'], columns='week').fillna(0).reset_index()
new_test_prophet_pivot = new_test_prophet.pivot(index=['username', 'course_id', 'action'], columns='week').fillna(0).reset_index()

In [224]:
new_train_prophet_pivot

username                             course_id   action count         \
week                                                               1      2   
0             5                 TsinghuaX/60510102X/_    close   1.0    0.0   
1             5                 TsinghuaX/60510102X/_    video  10.0    0.0   
2             5           TsinghuaX/70240183x/2015_T2    close   3.0    0.0   
3             5           TsinghuaX/70240183x/2015_T2  problem   1.0    0.0   
4             5           TsinghuaX/70240183x/2015_T2    video  39.0    0.0   
...         ...                                   ...      ...   ...    ...   
452406  7023562  course-v1:TsinghuaX+AP000008X+2017T1  problem   0.0   10.0   
452407  7023562  course-v1:TsinghuaX+AP000008X+2017T1    video   0.0  228.0   
452408  7036067  course-v1:TsinghuaX+AP000008X+2017T1    click   0.0    0.0   
452409  7036067  course-v1:TsinghuaX+AP000008X+2017T1  problem   0.0    0.0   
452410  7036067  course-v1:TsinghuaX+AP000008X+2017T1    video   0.0    0.0   

                            
week      3    4    5    6  
0       0.0  0.0  0.0  0.0  
1       0.0  0.0  0.0  0.0  
2       0.0  0.0  0.0  0.0  
3       0.0  0.0  0.0  0.0  
4       0.0  0.0  0.0  0.0  
...     ...  ...  ...  ...  
452406  0.0  0.0  0.0  0.0  
452407  0.0  0.0  0.0  0.0  
452408  7.0  0.0  0.0  0.0  
452409  8.0  0.0  0.0  0.0  
452410  5.0  0.0  0.0  0.0  

[452411 rows x 9 columns]

In [222]:
new_test_prophet_pivot

username                              course_id   action count       \
week                                                                1    2   
0             5    course-v1:Tsinghua+20150001+2015_T2    click  14.0  0.0   
1             5    course-v1:Tsinghua+20150001+2015_T2    close   3.0  0.0   
2             5    course-v1:Tsinghua+20150001+2015_T2    video  12.0  0.0   
3             5  course-v1:TsinghuaX+00740123X+2016_T1    click   0.0  0.0   
4             5  course-v1:TsinghuaX+00740123X+2016_T1    video   0.0  0.0   
...         ...                                    ...      ...   ...  ...   
193809  7006106   course-v1:TsinghuaX+AP000008X+2017T1    video   0.0  9.0   
193810  7008105   course-v1:TsinghuaX+AP000008X+2017T1    click   1.0  0.0   
193811  7008105   course-v1:TsinghuaX+AP000008X+2017T1  problem   3.0  0.0   
193812  7008105   course-v1:TsinghuaX+AP000008X+2017T1    video   1.0  0.0   
193813  7059814   course-v1:TsinghuaX+AP000008X+2017T1  problem   0.0  0.0   

                             
week      3     4    5    6  
0       0.0   0.0  0.0  0.0  
1       0.0   0.0  0.0  0.0  
2       0.0   0.0  0.0  0.0  
3       0.0   0.0  2.0  0.0  
4       0.0   0.0  5.0  0.0  
...     ...   ...  ...  ...  
193809  0.0   0.0  0.0  0.0  
193810  0.0   0.0  0.0  0.0  
193811  0.0   0.0  0.0  0.0  
193812  0.0   0.0  0.0  0.0  
193813  0.0  95.0  0.0  0.0  

[193814 rows x 9 columns]

In [235]:
train_prophet = new_train_prophet_pivot.drop(columns=['username', 'course_id', 'action']).T.reset_index().drop(columns=['level_0'])
train_prophet.rename(columns={'week': 'ds'}, inplace=True)

/Users/matteorusso/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [236]:
train_prophet

,ds,0,1,2,3,4,5,6,7,8,...,452401,452402,452403,452404,452405,452406,452407,452408,452409,452410
0,1,1.0,10.0,3.0,1.0,39.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,21.0,8.0,10.0,228.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,8.0,5.0
3,4,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,...,11.0,4.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,5.0,46.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Use Prophet library

In [242]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error

In [281]:
def prediction_prophet(userid, K, train_pivot):
    # select user to predict on
    train_pivot_copy = train_pivot.copy()
    train_pivot_copy.rename(columns={userid: 'y'}, inplace=True)
    all_users_in_course = [col for col in train_pivot_copy.columns if col != 'ds' and col != 'y']
    n_weeks = train_pivot_copy.shape[0]
    # change to date format for prophet to run
    train_pivot_copy['ds'] = pd.date_range(start="2022-01-01",end="2022-01-06") # these are dummy dates
    
    # create lists to be saved for later
    models = []
    y_true_running = []
    y_pred_running = []
    maes = []
    
    for t in range(2, n_weeks-1):
        # reduce training set to first t weeks
        train_pivot_t = train_pivot_copy.head(t)
        test_pivot_t = train_pivot_copy.tail(-t)
        
        train_pivot_t.columns = train_pivot_t.columns.astype(str)
        
        # create the model with K additional regressors (other students)
        model = Prophet()
        for other_userid in all_users_in_course[:K]:
            model.add_regressor(str(other_userid))
            
        print(train_pivot_t)
        
        # fit the model
        model.fit(train_pivot_t)
        models.append(model)
        
        # perform forecast
        future = model.make_future_dataframe(periods=test_pivot_t.shape[0])
        columns_to_keep = train_pivot_copy.columns[2:K+2]
        for_future = train_pivot_copy[columns_to_keep]
        futures = pd.concat((future, for_future), axis=1)
        forecast = model.predict(futures)
        
        # log results
        forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        y_true = test_pivot_t['y'].values
        y_true_running.append(y_true)
        y_pred = forecast['yhat'][t:].values
        y_pred_running.append(y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        maes.append(mae)
    
    return models, y_true_running, y_pred_running, maes

In [ ]:
maes_dict = {}
for userid in list(train_prophet.columns[1:]):
    print(userid)
    maes_list = []
    print(train_prophet.shape[1]-2)
    models, y_true_running, y_pred_running, maes = prediction_prophet(userid=userid, K=train_prophet.shape[1]-2, \
                                                                      train_pivot=train_prophet)
    maes_dict[userid] = maes

0
452410
          ds    y     1    2    3     4    5    6    7    8  ...  452401  \
0 2022-01-01  1.0  10.0  3.0  1.0  39.0  0.0  0.0  0.0  0.0  ...     0.0   
1 2022-01-02  0.0   0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...     0.0   

   452402  452403  452404  452405  452406  452407  452408  452409  452410  
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0  
1     0.0     0.0    21.0     8.0    10.0   228.0     0.0     0.0     0.0  

[2 rows x 452412 columns]


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
